In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
DS_FLUX_WGT.sel(igg=1).sel(g=1).to_dataframe();

In [14]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    
    


In [15]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    s_outputs_byg = 'Outputs by g-interval'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_outputs_byg))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [16]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 913ed72
molecule = {'co2': 0.0004}
band = [4]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD            CLIRAD `coolr_bands`           
            layer     coolrg                layer     coolrg
pressure                                                    
0.000312        1  66.811869                    1  32.440264
0.000750        2  30.192711                    2  27.263034
0.001052        3  28.255311                    3  28.586870
0.001476        4  25.341715                    4  28.631419
0.002070        5  22.174221                    5  27.126095
0.002904        6  19.226535                    6  24.105245
0.004074        7  16.722187                    7  20.043345
0.005714        8  14.724929                    8  15.923440
0.008015        9  13.199432                    9  12.752080
0.011243       10  12.055478                   10  11.086864
0.015771       11  11.189547                   11  10.692396
0.022122       12  10.518709                   12  10.857646
0.031031       13  10.089954                   13  11.100807
0.043528       14   9.446098                   14  10.727485
0.061057       15   8.347313                   15   9.445961
0.085645       16   7.340092                   16   7.903132
0.120136       17   6.662366                   17   6.650478
0.168516       18   6.248298                   18   5.960325
0.236378       19   6.084259                   19   5.897100
0.331549       20   6.196347                   20   6.301464
0.465100       21   6.809821                   21   7.155941
0.652400       22   6.510550                   22   6.897926
0.915100       23   5.072440                   23   5.261861
1.283650       24   3.910509                   24   3.728161
1.800600       25   3.126184                   25   2.822274
2.525700       26   2.505188                   26   2.447347
3.542800       27   1.954727                   27   2.213201
4.969550       28   1.571513                   28   1.839106
6.970850       29   1.401774                   29   1.494869
9.778100       30   1.262336                   30   1.286554
13.715850      31   1.074496                   31   1.241899
19.239350      32   0.841936                   32   1.082816
26.987250      33   0.698846                   33   0.833205
37.855300      34   0.648636                   34   0.653120
53.100050      35   0.551658                   35   0.579068
73.887500      36   0.434397                   36   0.526420
97.662500      37   0.339799                   37   0.399836
121.437500     38   0.247667                   38   0.239774
145.212500     39   0.170539                   39   0.117958
168.987500     40   0.126622                   40   0.063940
192.762500     41   0.095783                   41   0.042955
216.537500     42   0.070982                   42   0.034851
240.312500     43   0.047261                   43   0.028085
264.087500     44   0.012151                   44   0.006852
287.862500     45  -0.067821                   45  -0.070277
311.637500     46  -0.102537                   46  -0.102415
335.412500     47  -0.036092                   47  -0.028052
359.187500     48   0.019780                   48   0.027904
382.962500     49   0.036545                   49   0.038851
406.737500     50   0.044235                   50   0.043056
430.512500     51   0.048698                   51   0.046203
454.287500     52   0.051478                   52   0.047457
478.062500     53   0.053335                   53   0.049465
501.837500     54   0.054652                   54   0.050131
525.612500     55   0.055710                   55   0.050662
549.387500     56   0.056489                   56   0.050671
573.162500     57   0.056954                   57   0.050257
596.937500     58   0.056909                   58   0.049495
620.712500     59   0.056070                   59   0.047037
644.487500     60   0.054315                   60   0.045228
668.262500     61   0.051681                   61   0.041619
692.037500     62   0.048408                   62   0.037682
715.812500     63   0.044866                   63

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -14.290363   0.000000 -14.290363 -14.001680  9.886700e-07   
0.000624    2     -14.291621   0.006200 -14.285421 -14.002913  3.633129e-03   
0.000876    3     -14.292224   0.007703 -14.284522 -14.003424  4.956529e-03   
0.001229    4     -14.292966   0.009625 -14.283341 -14.004069  6.796470e-03   
0.001723    5     -14.293870   0.012014 -14.281856 -14.004876  9.281424e-03   
0.002417    6     -14.294965   0.014932 -14.280033 -14.005874  1.250958e-02   
0.003391    7     -14.296284   0.018469 -14.277816 -14.007097  1.651287e-02   
0.004757    8     -14.297864   0.022754 -14.275111 -14.008585  2.124350e-02   
0.006672    9     -14.299741   0.027972 -14.271769 -14.010377  2.664878e-02   
0.009359    10    -14.301953   0.034385 -14.267568 -14.012511  3.284178e-02   
0.013128    11    -14.304530   0.042344 -14.262186 -14.015018  4.029929e-02   
0.018415    12    -14.307484   0.052305 -14.255179 -14.017910  4.988847e-02   
0.025830    13    -14.310774   0.064835 -14.245939 -14.021140  6.265714e-02   
0.036232    14    -14.314111   0.080606 -14.233506 -14.024408  7.960515e-02   
0.050823    15    -14.317192   0.100013 -14.217179 -14.027409  1.011487e-01   
0.071291    16    -14.320175   0.123234 -14.196941 -14.030268  1.269119e-01   
0.100000    17    -14.323171   0.151192 -14.171978 -14.033039  1.565631e-01   
0.140271    18    -14.325843   0.185646 -14.140196 -14.035328  1.905807e-01   
0.196760    19    -14.327477   0.229092 -14.098386 -14.036343  2.314838e-01   
0.275997    20    -14.326558   0.285280 -14.041278 -14.034520  2.850177e-01   
0.387100    21    -14.319311   0.359583 -13.959728 -14.026189  3.596279e-01   
0.543100    22    -14.292482   0.458595 -13.833887 -13.998539  4.642292e-01   
0.761700    23    -14.238761   0.573463 -13.665298 -13.945268  5.895967e-01   
1.068500    24    -14.170056   0.689104 -13.480952 -13.877753  7.133314e-01   
1.498800    25    -14.092345   0.810720 -13.281626 -13.801198  8.268277e-01   
2.102400    26    -14.004916   0.946815 -13.058101 -13.713800  9.412457e-01   
2.949000    27    -13.908836   1.101970 -12.806866 -13.616221  1.089126e+00   
4.136600    28    -13.814071   1.282194 -12.531876 -13.518028  1.302318e+00   
5.802500    29    -13.730627   1.508870 -12.221757 -13.430296  1.577548e+00   
8.139200    30    -13.651048   1.817300 -11.833748 -13.344436  1.905508e+00   
11.417000   31    -13.570396   2.226787 -11.343610 -13.255247  2.315913e+00   
16.014700   32    -13.503915   2.745508 -10.758406 -13.180596  2.917708e+00   
22.464000   33    -13.505078   3.389882 -10.115196 -13.180851  3.745281e+00   
31.510500   34    -13.601566   4.235268  -9.366298 -13.286441  4.743846e+00   
44.200100   35    -13.746123   5.354838  -8.391286 -13.446803  5.886062e+00   
62.000000   36    -13.916374   6.688273  -7.228101 -13.635545  7.295909e+00   
85.775000   37    -14.071263   8.066562  -6.004701 -13.804076  8.947159e+00   
109.550000  38    -14.143895   9.096176  -5.047719 -13.880283  1.014955e+01   
133.325000  39    -14.172398   9.822188  -4.350210 -13.905529  1.085014e+01   
157.100000  40    -14.205691  10.335774  -3.869917 -13.934022  1.121088e+01   
180.875000  41    -14.247556  10.734246  -3.513310 -13.968805  1.142575e+01   
204.650000  42    -14.300698  11.057142  -3.243556 -14.010951  1.158889e+01   
228.425000  43    -14.369705  11.326057  -3.043648 -14.062691  1.173879e+01   
252.200000  44    -14.464683  11.554138  -2.910545 -14.130291  1.188549e+01   
275.975000  45    -14.627340  11.751017  -2.876323 -14.256873  1.203138e+01   
299.750000  46    -15.017750  11.950421  -3.067329 -14.629598  1.220616e+01   
323.525000  47    -15.688954  12.332850  -3.356104 -15.317059  1.260516e+01   
347.300000  48    -16.469042  13.011291  -3.457751 -16.128499  1.333759e+01

In [17]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')